In [ ]:
import collections
from datetime import datetime
import functools
import itertools
import os
import pathlib
import requests
import string
import sys
import typing
import zipfile

import IPython
import numpy as np
import pandas as pd
import pypandoc

from tqdm.notebook import tqdm_notebook

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_pipeline.utils import remove_all_from_dir, get_excel_column_name
from data_pipeline.etl.sources.census.etl_utils import get_state_information

# Turn on TQDM for pandas so that we can have progress bars when running `apply`.
tqdm_notebook.pandas()

In [ ]:
# Suppress scientific notation in pandas (this shows up for census tract IDs)
pd.options.display.float_format = "{:.2f}".format

# Set some global parameters
DATA_DIR = pathlib.Path.cwd().parent / "data"
TEMP_DATA_DIR = DATA_DIR / "tmp"
COMPARISON_OUTPUTS_DIR = DATA_DIR / "comparison_outputs"

## I (Vincent) created this manually locally. Will need to change potentially when putting into official ETL scripts
GEOCORR_DATA_DIR = DATA_DIR / "geocorr"

# Make the dirs if they don't exist
TEMP_DATA_DIR.mkdir(parents=True, exist_ok=True)
COMPARISON_OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

CEJST_PRIORITY_COMMUNITY_THRESHOLD = 0.75

# Name fields using variables. (This makes it easy to reference the same fields frequently without using strings
# and introducing the risk of misspelling the field name.)

GEOID_FIELD_NAME = "GEOID10"
GEOID_TRACT_FIELD_NAME = "GEOID10_TRACT"
GEOID_STATE_FIELD_NAME = "GEOID10_STATE"
GEOID_CBG_FIELD_NAME = "GEOID10_CBG"
COUNTRY_FIELD_NAME = "Country"
CENSUS_BLOCK_GROUP_POPULATION_FIELD = "Total population"

CEJST_SCORE_FIELD = "cejst_score"
CEJST_PERCENTILE_FIELD = "cejst_percentile"
CEJST_PRIORITY_COMMUNITY_FIELD = "cejst_priority_community"

# Define some suffixes
POPULATION_SUFFIX = " (priority population)"

## Mapping Census Block Group to Urban and Rural Indicators using Geocorr Data

The end result is a dataframe `urban_rural_map`

In [ ]:
geocorr_urban_rural_map = pd.read_csv(
    os.path.join(GEOCORR_DATA_DIR, 'geocorr2014_2125804280.csv'),
    encoding = "ISO-8859-1",
    skiprows=[1],
    dtype='str',
)

geocorr_urban_rural_map['pop10'] = pd.to_numeric(geocorr_urban_rural_map['pop10'])
geocorr_urban_rural_map['afact'] = pd.to_numeric(geocorr_urban_rural_map['afact'])

geocorr_urban_rural_map[GEOID_TRACT_FIELD_NAME] = geocorr_urban_rural_map['county'] + geocorr_urban_rural_map['tract'] # + geocorr_urban_rural_map['bg']
geocorr_urban_rural_map[GEOID_TRACT_FIELD_NAME] = geocorr_urban_rural_map[GEOID_TRACT_FIELD_NAME].str.replace('.', '', regex=False)

In [ ]:
geocorr_urban_rural_map[GEOID_TRACT_FIELD_NAME].str.len().value_counts()

We want to see that the length of the derived Census Block Group is always 12 digits. Census Tracts are always 11 digits

In [ ]:
geocorr_urban_rural_map = geocorr_urban_rural_map[[
    GEOID_TRACT_FIELD_NAME,
    'ur',
    'ua',
    'cntyname',
    'uaname',
    'pop10',
    'afact'
]]

Checking Primary Key

In [ ]:
geocorr_urban_rural_map.groupby([GEOID_TRACT_FIELD_NAME, 'ur', 'ua'], dropna=False).size().sort_values(ascending=False)

In [ ]:
geocorr_urban_rural_map.loc[geocorr_urban_rural_map[GEOID_TRACT_FIELD_NAME] == '36117020302']

In [ ]:
total_geo_population = geocorr_urban_rural_map.groupby(GEOID_TRACT_FIELD_NAME).agg({'pop10': np.sum}).reset_index()
total_geo_population.rename(columns={'pop10': 'total_population'}, inplace=True)
total_geo_population.head()

In [ ]:
geocorr_urban_rural_with_total_pop_map = geocorr_urban_rural_map.groupby([GEOID_TRACT_FIELD_NAME, 'ur']).agg({'pop10': np.sum}).reset_index()
geocorr_urban_rural_with_total_pop_map = geocorr_urban_rural_with_total_pop_map.merge(total_geo_population, how='inner', on=GEOID_TRACT_FIELD_NAME)
geocorr_urban_rural_with_total_pop_map.head()

In [ ]:
geocorr_urban_rural_with_total_pop_map['afact'] = geocorr_urban_rural_with_total_pop_map['pop10'] / geocorr_urban_rural_with_total_pop_map['total_population']

In [ ]:
geocorr_urban_rural_with_total_pop_map.head()

In [ ]:
geocorr_urban_rural_with_total_pop_map.loc[geocorr_urban_rural_with_total_pop_map[GEOID_TRACT_FIELD_NAME] == '01001020200']

In [ ]:
urban_rural_map = geocorr_urban_rural_with_total_pop_map.pivot(index=GEOID_TRACT_FIELD_NAME, columns='ur', values=['pop10', 'afact'])
urban_rural_map.columns = ['_'.join(col).strip() for col in urban_rural_map.columns.values]
urban_rural_map.reset_index(inplace=True)
urban_rural_map['urban_heuristic_flag'] = 0
mask = urban_rural_map['afact_U'] >= 0.5
urban_rural_map.loc[mask, 'urban_heuristic_flag'] = 1

In [ ]:
urban_rural_map.rename(
    columns={
        'pop10_R': 'population_in_rural_areas',
        'pop10_U': 'population_in_urban_areas',
        'afact_R': 'perc_population_in_rural_areas',
        'afact_U': 'perc_population_in_urban_areas',
    }, 
    inplace=True)

In [ ]:
urban_rural_map.head(5)

In [ ]:
urban_rural_map.to_csv(
    path_or_buf=GEOCORR_DATA_DIR / "urban_rural_map.csv", na_rep="", index=False
)